# Per-Outlet Topic Modeling & Cross-Outlet Comparison

This notebook runs BERTopic **independently per newspaper outlet**, then cross-identifies matching topics
and compares how outlets frame the same topics differently.

## Approach
1. Load pre-computed embeddings (same `all-mpnet-base-v2` space across all outlets)
2. Fit separate BERTopic models per outlet with adjusted parameters for smaller corpora
3. Match topics across outlets using a 3-method ensemble (centroid similarity, keyword overlap, c-TF-IDF)
4. Use the Hungarian algorithm for optimal 1-to-1 matching per outlet pair
5. Group matched topics transitively via Union-Find
6. Visualize coverage differences, keyword divergence, and selection bias signals

## 1. Setup & Data Loading

In [1]:
import sys
import warnings
import numpy as np
import pandas as pd
from collections import defaultdict
from itertools import combinations

from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cosine
from scipy.sparse import csr_matrix

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sentence_transformers import SentenceTransformer

warnings.filterwarnings('ignore')
sys.path.append('..')

from src.db import get_db
from src.versions import list_versions, get_version, get_version_config
from dashboard.components.source_mapping import SOURCE_NAMES, SOURCE_COLORS

print("All imports successful")

All imports successful


In [2]:
# Configuration
OUTLETS = ["dailynews_en", "themorning_en", "ft_en", "island_en"]
RANDOM_SEED = 42
EMBEDDING_MODEL = "all-mpnet-base-v2"

# BERTopic parameters adjusted for smaller per-outlet corpora (~500 articles each)
MIN_TOPIC_SIZE = 5
MIN_DF = 2
UMAP_N_NEIGHBORS = 10

# Cross-outlet matching parameters
WEIGHT_CENTROID = 0.50
WEIGHT_CTFIDF = 0.25
WEIGHT_KEYWORD = 0.25
MATCH_THRESHOLD = 0.45
KEYWORD_TOP_N = 10

# Find a topic version whose embeddings use all-mpnet-base-v2
topic_versions = list_versions(analysis_type='topics')

matching_versions = []
for v in topic_versions:
    cfg = v.get("configuration", {})
    model_name = cfg.get("embeddings", {}).get("model", "")
    has_embeddings = (v.get("pipeline_status") or {}).get("embeddings", False)
    matching_versions.append({
        **v,
        "emb_model": model_name,
        "has_embeddings": has_embeddings,
    })
    status = "MATCH" if model_name == EMBEDDING_MODEL and has_embeddings else "skip"
    print(f"  [{status}] {v['name']} ({v['id'][:8]}...) "
          f"model={model_name or 'N/A'}, embeddings={has_embeddings}")

# Filter to versions with the correct embedding model AND completed embeddings
usable = [v for v in matching_versions
          if v["emb_model"] == EMBEDDING_MODEL and v["has_embeddings"]]

if not usable:
    raise RuntimeError(
        f"No topic version found with embedding model '{EMBEDDING_MODEL}' "
        f"and completed embeddings. Available versions:\n"
        + "\n".join(f"  {v['name']}: model={v['emb_model']}, embeddings={v['has_embeddings']}"
                    for v in matching_versions)
    )

version_id = usable[0]["id"]
print(f"\nUsing version: {usable[0]['name']} ({version_id[:8]}...)")
print(f"  Embedding model: {usable[0]['emb_model']}")
print(f"  Analysis type: {usable[0]['analysis_type']}")
print(f"  Complete: {usable[0]['is_complete']}")

  [skip] v0130-1438 (5559335a...) model=google/embeddinggemma-300m, embeddings=True
  [MATCH] v0128-2047 (6a349181...) model=all-mpnet-base-v2, embeddings=True
  [MATCH] v0128-1606 (a2571328...) model=all-mpnet-base-v2, embeddings=True

Using version: v0128-2047 (6a349181...)
  Embedding model: all-mpnet-base-v2
  Analysis type: topics
  Complete: True


In [3]:
# Load all embeddings and split by outlet
print("Loading embeddings from database...")
with get_db() as db:
    all_data = db.get_all_embeddings(result_version_id=version_id)

print(f"Total articles loaded: {len(all_data)}")

# Split by source_id
outlet_data = {}
for outlet in OUTLETS:
    records = [d for d in all_data if d['source_id'] == outlet]
    outlet_data[outlet] = {
        'documents': [f"{d['title']}\n\n{d['content'][:8000]}" for d in records],
        'embeddings': np.array([d['embedding'] for d in records]),
        'article_ids': [d['article_id'] for d in records],
        'titles': [d['title'] for d in records],
    }
    print(f"  {SOURCE_NAMES[outlet]}: {len(records)} articles")

del all_data  # free memory

Loading embeddings from database...
Total articles loaded: 1657
  Daily News: 502 articles
  The Morning: 491 articles
  Daily FT: 380 articles
  The Island: 284 articles


## 2. Per-Outlet BERTopic Modeling

In [4]:
# Load embedding model once, share across all fits
print(f"Loading SentenceTransformer model ({EMBEDDING_MODEL})...")
embedding_model = SentenceTransformer(EMBEDDING_MODEL)

custom_stop_words = sorted(set(ENGLISH_STOP_WORDS) | {"sri", "lanka", "lankan"})

# Fit BERTopic per outlet
outlet_results = {}

for outlet in OUTLETS:
    name = SOURCE_NAMES[outlet]
    data = outlet_data[outlet]
    n_docs = len(data['documents'])
    print(f"\n{'='*60}")
    print(f"Fitting BERTopic for {name} ({n_docs} articles)")
    print(f"{'='*60}")

    umap_model = UMAP(
        n_neighbors=UMAP_N_NEIGHBORS,
        n_components=5,
        min_dist=0.0,
        metric='cosine',
        random_state=RANDOM_SEED,
    )

    hdbscan_model = HDBSCAN(
        min_cluster_size=MIN_TOPIC_SIZE,
        metric='euclidean',
        cluster_selection_method='eom',
        prediction_data=True,
        core_dist_n_jobs=1,
    )

    vectorizer = CountVectorizer(
        ngram_range=(1, 3),
        stop_words=custom_stop_words,
        min_df=MIN_DF,
    )

    model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer,
        representation_model=KeyBERTInspired(),
        verbose=False,
    )

    topics, probs = model.fit_transform(data['documents'], data['embeddings'])
    topic_info = model.get_topic_info()

    n_topics = len(topic_info[topic_info['Topic'] != -1])
    n_outliers = sum(1 for t in topics if t == -1)
    print(f"  Topics: {n_topics}, Outliers: {n_outliers}/{n_docs}")

    outlet_results[outlet] = {
        'model': model,
        'topics': topics,
        'probs': probs,
        'topic_info': topic_info,
        'documents': data['documents'],
        'embeddings': data['embeddings'],
        'article_ids': data['article_ids'],
    }

print("\nAll models fitted successfully.")

Loading SentenceTransformer model (all-mpnet-base-v2)...

Fitting BERTopic for Daily News (502 articles)
  Topics: 30, Outliers: 90/502

Fitting BERTopic for The Morning (491 articles)
  Topics: 27, Outliers: 72/491

Fitting BERTopic for Daily FT (380 articles)
  Topics: 22, Outliers: 96/380

Fitting BERTopic for The Island (284 articles)
  Topics: 13, Outliers: 52/284

All models fitted successfully.


In [5]:
# Summary table
summary_rows = []
for outlet in OUTLETS:
    r = outlet_results[outlet]
    ti = r['topic_info']
    n_topics = len(ti[ti['Topic'] != -1])
    n_outliers = sum(1 for t in r['topics'] if t == -1)
    summary_rows.append({
        'Outlet': SOURCE_NAMES[outlet],
        'Articles': len(r['documents']),
        'Topics': n_topics,
        'Outliers': n_outliers,
        'Outlier %': f"{100*n_outliers/len(r['documents']):.1f}%",
    })

summary_df = pd.DataFrame(summary_rows)
print("Per-Outlet Topic Modeling Summary")
display(summary_df)

Per-Outlet Topic Modeling Summary


,Outlet,Articles,Topics,Outliers,Outlier %
0,Daily News,502,30,90,17.9%
1,The Morning,491,27,72,14.7%
2,Daily FT,380,22,96,25.3%
3,The Island,284,13,52,18.3%


## 3. Per-Outlet Topic Overview

In [7]:
# List all topics per outlet with top 30 keywords/ngrams
TOP_N_WORDS = 30

for outlet in OUTLETS:
    name = SOURCE_NAMES[outlet]
    r = outlet_results[outlet]
    ti = r['topic_info']
    ti_topics = ti[ti['Topic'] != -1].sort_values('Count', ascending=False)

    print(f"\n{'='*100}")
    print(f"  {name} — {len(ti_topics)} topics, {len(r['documents'])} articles")
    print(f"{'='*100}")

    for _, row in ti_topics.iterrows():
        tid = row['Topic']
        count = row['Count']
        words = r['model'].get_topic(tid)
        if words:
            keyword_str = ', '.join(w for w, _ in words[:TOP_N_WORDS])
        else:
            keyword_str = '(no keywords)'
        print(f"\n  Topic {tid} ({count} articles)")
        print(f"    {keyword_str}")


  Daily News — 30 topics, 502 articles

  Topic 0 (32 articles)
    affected cyclone ditwah, affected cyclone, cyclone ditwah, cyclone, ceylon, ditwah, disaster, affected businesses, impacted, affected

  Topic 1 (31 articles)
    cyclone ditwah, cyclone, kumara dissanayake, indian external affairs, anura kumara dissanayake, colombo, india, ditwah, president anura kumara, affairs minister dr

  Topic 2 (24 articles)
    cyclone ditwah, disaster management, cyclones, floods, tsunami, cyclone, evacuation, rainfall, natural disasters, flood

  Topic 3 (19 articles)
    cyclone ditwah, cyclone, port city colombo, city colombo, colombo, ditwah, flood, relief centres, disaster, affected

  Topic 4 (19 articles)
    relief communities affected, affected cyclone ditwah, affected cyclone, communities affected cyclone, relief communities, cyclone ditwah, cyclone, communities affected, government rebuilding fund, established provide relief

  Topic 5 (18 articles)
    cyclonic storm ditwah, cycl

In [8]:
# Intra-model topic similarity heatmaps (c-TF-IDF cosine similarity)
for outlet in OUTLETS:
    name = SOURCE_NAMES[outlet]
    model = outlet_results[outlet]['model']
    n_topics = len(model.get_topic_info()[model.get_topic_info()['Topic'] != -1])
    fig = model.visualize_heatmap(top_n_topics=n_topics, title=f"{name} — Inter-Topic Similarity")
    fig.show()

## 4. Cross-Outlet Topic Matching (3 Methods)

In [ ]:
def get_topic_ids(outlet):
    """Get non-outlier topic IDs for an outlet."""
    ti = outlet_results[outlet]['topic_info']
    return sorted(ti[ti['Topic'] != -1]['Topic'].tolist())


def compute_topic_centroids(outlet):
    """Compute mean embedding per topic."""
    r = outlet_results[outlet]
    centroids = {}
    for tid in get_topic_ids(outlet):
        mask = np.array(r['topics']) == tid
        if mask.sum() > 0:
            centroids[tid] = r['embeddings'][mask].mean(axis=0)
    return centroids


# Method 1: Centroid embedding similarity
print("Computing topic centroids per outlet...")
outlet_centroids = {o: compute_topic_centroids(o) for o in OUTLETS}


def centroid_similarity(outlet_a, tid_a, outlet_b, tid_b):
    """Cosine similarity between two topic centroids."""
    ca = outlet_centroids[outlet_a].get(tid_a)
    cb = outlet_centroids[outlet_b].get(tid_b)
    if ca is None or cb is None:
        return 0.0
    return float(1.0 - cosine(ca, cb))


# Method 2: Keyword Jaccard similarity
def keyword_jaccard(outlet_a, tid_a, outlet_b, tid_b, top_n=KEYWORD_TOP_N):
    """Jaccard similarity of top-N keyword sets."""
    words_a = outlet_results[outlet_a]['model'].get_topic(tid_a)
    words_b = outlet_results[outlet_b]['model'].get_topic(tid_b)
    if not words_a or not words_b:
        return 0.0
    set_a = set(w for w, _ in words_a[:top_n])
    set_b = set(w for w, _ in words_b[:top_n])
    intersection = len(set_a & set_b)
    union = len(set_a | set_b)
    return intersection / union if union > 0 else 0.0


# Method 3: c-TF-IDF similarity in union vocabulary space
def ctfidf_similarity_matrix(outlet_a, outlet_b):
    """Cosine similarity between c-TF-IDF vectors in a shared vocabulary space."""
    model_a = outlet_results[outlet_a]['model']
    model_b = outlet_results[outlet_b]['model']

    # Get vocabularies
    vocab_a = model_a.vectorizer_model.get_feature_names_out()
    vocab_b = model_b.vectorizer_model.get_feature_names_out()

    # Build union vocabulary
    union_vocab = sorted(set(vocab_a) | set(vocab_b))
    word_to_idx = {w: i for i, w in enumerate(union_vocab)}

    def project_ctfidf(model, vocab):
        """Project c-TF-IDF matrix into union vocabulary space."""
        ctfidf = model.c_tf_idf_
        topic_ids = sorted([t for t in model.topic_labels_.keys() if t != -1])
        # BERTopic c_tf_idf_ rows: row 0 = outlier topic (-1), row 1+ = topics 0, 1, ...
        projected = np.zeros((len(topic_ids), len(union_vocab)))
        for i, tid in enumerate(topic_ids):
            row_idx = tid + 1  # offset for outlier row
            if row_idx >= ctfidf.shape[0]:
                continue
            row = ctfidf[row_idx]
            if hasattr(row, 'toarray'):
                row = row.toarray().flatten()
            for j, word in enumerate(vocab):
                if j < len(row) and word in word_to_idx:
                    projected[i, word_to_idx[word]] = row[j]
        return projected, topic_ids

    proj_a, tids_a = project_ctfidf(model_a, vocab_a)
    proj_b, tids_b = project_ctfidf(model_b, vocab_b)

    # Compute pairwise cosine similarity
    sim_matrix = np.zeros((len(tids_a), len(tids_b)))
    for i in range(len(tids_a)):
        norm_a = np.linalg.norm(proj_a[i])
        if norm_a == 0:
            continue
        for j in range(len(tids_b)):
            norm_b = np.linalg.norm(proj_b[j])
            if norm_b == 0:
                continue
            sim_matrix[i, j] = np.dot(proj_a[i], proj_b[j]) / (norm_a * norm_b)

    return sim_matrix, tids_a, tids_b


print("Matching functions defined.")

## 5. Topic Matching via Hungarian Algorithm

In [ ]:
# Compute pairwise matches for all 6 outlet pairs
outlet_pairs = list(combinations(OUTLETS, 2))
pairwise_matches = {}  # (outlet_a, outlet_b) -> list of (tid_a, tid_b, score)
pairwise_centroid_sims = {}  # for heatmap visualization

for outlet_a, outlet_b in outlet_pairs:
    name_a = SOURCE_NAMES[outlet_a]
    name_b = SOURCE_NAMES[outlet_b]
    print(f"\nMatching: {name_a} <-> {name_b}")

    tids_a = get_topic_ids(outlet_a)
    tids_b = get_topic_ids(outlet_b)

    if not tids_a or not tids_b:
        print("  Skipping: one outlet has no topics.")
        pairwise_matches[(outlet_a, outlet_b)] = []
        continue

    # Build ensemble score matrix
    n_a, n_b = len(tids_a), len(tids_b)

    # Method 1: Centroid similarity
    centroid_sim = np.zeros((n_a, n_b))
    for i, ta in enumerate(tids_a):
        for j, tb in enumerate(tids_b):
            centroid_sim[i, j] = centroid_similarity(outlet_a, ta, outlet_b, tb)

    pairwise_centroid_sims[(outlet_a, outlet_b)] = (centroid_sim, tids_a, tids_b)

    # Method 2: Keyword Jaccard
    keyword_sim = np.zeros((n_a, n_b))
    for i, ta in enumerate(tids_a):
        for j, tb in enumerate(tids_b):
            keyword_sim[i, j] = keyword_jaccard(outlet_a, ta, outlet_b, tb)

    # Method 3: c-TF-IDF similarity
    ctfidf_sim, ctfidf_tids_a, ctfidf_tids_b = ctfidf_similarity_matrix(outlet_a, outlet_b)

    # Align c-TF-IDF matrix to match tids_a/tids_b ordering
    ctfidf_aligned = np.zeros((n_a, n_b))
    tid_a_idx = {t: i for i, t in enumerate(ctfidf_tids_a)}
    tid_b_idx = {t: i for i, t in enumerate(ctfidf_tids_b)}
    for i, ta in enumerate(tids_a):
        for j, tb in enumerate(tids_b):
            if ta in tid_a_idx and tb in tid_b_idx:
                ctfidf_aligned[i, j] = ctfidf_sim[tid_a_idx[ta], tid_b_idx[tb]]

    # Weighted ensemble
    ensemble = (
        WEIGHT_CENTROID * centroid_sim
        + WEIGHT_CTFIDF * ctfidf_aligned
        + WEIGHT_KEYWORD * keyword_sim
    )

    # Hungarian algorithm (minimize cost = maximize similarity)
    cost = 1.0 - ensemble
    row_ind, col_ind = linear_sum_assignment(cost)

    matches = []
    for r, c in zip(row_ind, col_ind):
        score = ensemble[r, c]
        if score >= MATCH_THRESHOLD:
            matches.append((tids_a[r], tids_b[c], float(score)))
            print(f"  Match: T{tids_a[r]} <-> T{tids_b[c]} (score: {score:.3f})")

    pairwise_matches[(outlet_a, outlet_b)] = matches
    print(f"  Total matches: {len(matches)} (threshold: {MATCH_THRESHOLD})")

# Show ensemble score distribution for threshold calibration
all_scores = []
for matches in pairwise_matches.values():
    all_scores.extend([s for _, _, s in matches])

if all_scores:
    fig = px.histogram(
        x=all_scores, nbins=30,
        title='Ensemble Match Score Distribution (matched pairs only)',
        labels={'x': 'Ensemble Score', 'y': 'Count'},
    )
    fig.add_vline(x=MATCH_THRESHOLD, line_dash='dash', line_color='red',
                  annotation_text=f'Threshold={MATCH_THRESHOLD}')
    fig.update_layout(height=350)
    fig.show()

## 6. Transitive Topic Grouping (Union-Find)

In [ ]:
# Union-Find data structure
class UnionFind:
    def __init__(self):
        self.parent = {}
        self.rank = {}

    def find(self, x):
        if x not in self.parent:
            self.parent[x] = x
            self.rank[x] = 0
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        rx, ry = self.find(x), self.find(y)
        if rx == ry:
            return
        if self.rank[rx] < self.rank[ry]:
            rx, ry = ry, rx
        self.parent[ry] = rx
        if self.rank[rx] == self.rank[ry]:
            self.rank[rx] += 1


# Build union-find from pairwise matches
# Keys are (outlet, topic_id) tuples
uf = UnionFind()

# Register all topics
for outlet in OUTLETS:
    for tid in get_topic_ids(outlet):
        uf.find((outlet, tid))

# Union matched topics
for (outlet_a, outlet_b), matches in pairwise_matches.items():
    for tid_a, tid_b, score in matches:
        uf.union((outlet_a, tid_a), (outlet_b, tid_b))

# Extract groups
groups = defaultdict(list)
for outlet in OUTLETS:
    for tid in get_topic_ids(outlet):
        root = uf.find((outlet, tid))
        groups[root].append((outlet, tid))

# Classify groups
topic_groups = []
for group_id, (root, members) in enumerate(sorted(groups.items(), key=lambda x: -len(x[1]))):
    outlets_in_group = set(o for o, _ in members)
    # Build representative label from keywords across members
    all_keywords = []
    for o, tid in members:
        words = outlet_results[o]['model'].get_topic(tid)
        if words:
            all_keywords.extend([w for w, _ in words[:5]])
    # Most common keywords as group label
    from collections import Counter
    keyword_counts = Counter(all_keywords)
    top_keywords = [w for w, _ in keyword_counts.most_common(5)]
    label = ', '.join(top_keywords) if top_keywords else 'Unknown'

    topic_groups.append({
        'group_id': group_id,
        'label': label,
        'members': members,
        'n_outlets': len(outlets_in_group),
        'outlets': outlets_in_group,
        'is_shared': len(outlets_in_group) >= 2,
    })

shared_groups = [g for g in topic_groups if g['is_shared']]
unique_groups = [g for g in topic_groups if not g['is_shared']]

print(f"Total topic groups: {len(topic_groups)}")
print(f"  Shared (2+ outlets): {len(shared_groups)}")
print(f"  Unique (1 outlet):   {len(unique_groups)}")

print(f"\nShared Topic Groups:")
for g in shared_groups:
    outlet_names = [SOURCE_NAMES[o] for o in sorted(g['outlets'])]
    print(f"  Group {g['group_id']}: [{', '.join(outlet_names)}] - {g['label']}")

# Distribution by outlet count
dist = Counter(g['n_outlets'] for g in topic_groups)
print(f"\nGroups by outlet count:")
for n in sorted(dist.keys(), reverse=True):
    print(f"  In {n} outlet(s): {dist[n]} groups")

## 7. Visualizations

### 7a. Cross-Outlet Similarity Heatmaps

In [ ]:
# One heatmap per outlet pair showing centroid cosine similarity
for (outlet_a, outlet_b), (sim_matrix, tids_a, tids_b) in pairwise_centroid_sims.items():
    name_a = SOURCE_NAMES[outlet_a]
    name_b = SOURCE_NAMES[outlet_b]

    # Build topic labels
    def topic_label(outlet, tid):
        words = outlet_results[outlet]['model'].get_topic(tid)
        kw = ', '.join([w for w, _ in words[:3]]) if words else ''
        return f"T{tid}: {kw}"

    labels_a = [topic_label(outlet_a, t) for t in tids_a]
    labels_b = [topic_label(outlet_b, t) for t in tids_b]

    fig = px.imshow(
        sim_matrix,
        x=labels_b,
        y=labels_a,
        color_continuous_scale='RdYlGn',
        zmin=0, zmax=1,
        labels=dict(x=name_b, y=name_a, color='Cosine Sim'),
        title=f'Topic Centroid Similarity: {name_a} vs {name_b}',
    )
    fig.update_layout(height=500, width=800)
    fig.show()

### 7b. Keyword Divergence Analysis

In [ ]:
# For shared topic groups, show shared vs distinctive keywords per outlet
for g in shared_groups[:8]:  # limit to first 8 groups
    group_label = g['label']
    members = g['members']

    # Collect keywords per outlet in this group
    outlet_keywords = {}
    for o, tid in members:
        words = outlet_results[o]['model'].get_topic(tid)
        if words:
            outlet_keywords[SOURCE_NAMES[o]] = {w: s for w, s in words[:KEYWORD_TOP_N]}

    if len(outlet_keywords) < 2:
        continue

    # Find shared and distinctive keywords
    all_words = set()
    for kw_dict in outlet_keywords.values():
        all_words |= set(kw_dict.keys())

    shared_kws = set.intersection(*[set(kw.keys()) for kw in outlet_keywords.values()])

    # Build figure: side-by-side bars for each outlet
    fig = go.Figure()
    outlet_names = sorted(outlet_keywords.keys())

    for oname in outlet_names:
        kw_dict = outlet_keywords[oname]
        # Sort by score descending
        sorted_kw = sorted(kw_dict.items(), key=lambda x: x[1], reverse=True)
        words = [w for w, _ in sorted_kw]
        scores = [s for _, s in sorted_kw]
        colors = [SOURCE_COLORS[oname] if w not in shared_kws else '#888888' for w in words]

        fig.add_trace(go.Bar(
            name=oname,
            y=words[::-1],
            x=scores[::-1],
            orientation='h',
            marker_color=colors[::-1],
        ))

    fig.update_layout(
        title=f'Keyword Divergence: Group "{group_label}"<br><sub>Grey = shared keywords, colored = distinctive</sub>',
        barmode='group',
        height=400,
        xaxis_title='Keyword Score',
    )
    fig.show()

### 7c. Coverage Volume Comparison

In [ ]:
# Grouped bar chart: article counts per outlet per matched topic group
coverage_rows = []
for g in shared_groups:
    for outlet in OUTLETS:
        # Count articles in this group for this outlet
        member_tids = [tid for o, tid in g['members'] if o == outlet]
        count = 0
        if member_tids:
            topics_arr = np.array(outlet_results[outlet]['topics'])
            for tid in member_tids:
                count += int((topics_arr == tid).sum())
        coverage_rows.append({
            'Group': f"G{g['group_id']}: {g['label'][:30]}",
            'group_id': g['group_id'],
            'Outlet': SOURCE_NAMES[outlet],
            'Articles': count,
        })

coverage_df = pd.DataFrame(coverage_rows)

fig = px.bar(
    coverage_df,
    x='Group',
    y='Articles',
    color='Outlet',
    barmode='group',
    color_discrete_map=SOURCE_COLORS,
    title='Article Coverage per Outlet per Shared Topic Group',
)
fig.update_layout(height=500, xaxis_tickangle=-30)
fig.show()

### 7d. Topic Proportion Heatmap

In [ ]:
# % of each outlet's coverage devoted to each topic group, sorted by variance
prop_rows = []
for g in shared_groups:
    row = {'Group': f"G{g['group_id']}: {g['label'][:35]}"}
    proportions = []
    for outlet in OUTLETS:
        member_tids = [tid for o, tid in g['members'] if o == outlet]
        total_articles = len(outlet_results[outlet]['documents'])
        count = 0
        if member_tids:
            topics_arr = np.array(outlet_results[outlet]['topics'])
            for tid in member_tids:
                count += int((topics_arr == tid).sum())
        prop = count / total_articles if total_articles > 0 else 0
        row[SOURCE_NAMES[outlet]] = prop
        proportions.append(prop)
    row['variance'] = np.var(proportions)
    prop_rows.append(row)

prop_df = pd.DataFrame(prop_rows).sort_values('variance', ascending=False)

outlet_names = [SOURCE_NAMES[o] for o in OUTLETS]
heat_data = prop_df[outlet_names].values

fig = px.imshow(
    heat_data,
    x=outlet_names,
    y=prop_df['Group'].tolist(),
    color_continuous_scale='YlOrRd',
    labels=dict(color='Proportion'),
    title='Topic Coverage Proportions per Outlet<br><sub>Sorted by variance (highest = strongest selection bias signal)</sub>',
    text_auto='.1%',
)
fig.update_layout(height=max(400, len(prop_df) * 35), width=700)
fig.show()

### 7e. Sankey Diagram

In [ ]:
# Sankey diagram showing topic correspondences across outlets
# Nodes: each outlet's topics; Links: matched pairs

node_labels = []
node_colors = []
node_map = {}  # (outlet, tid) -> node index

for outlet in OUTLETS:
    color = SOURCE_COLORS[SOURCE_NAMES[outlet]]
    for tid in get_topic_ids(outlet):
        words = outlet_results[outlet]['model'].get_topic(tid)
        kw = ', '.join([w for w, _ in words[:3]]) if words else f'T{tid}'
        label = f"{SOURCE_NAMES[outlet]}\nT{tid}: {kw}"
        node_map[(outlet, tid)] = len(node_labels)
        node_labels.append(label)
        node_colors.append(color)

# Links from matches
link_source = []
link_target = []
link_value = []
link_color = []

for (outlet_a, outlet_b), matches in pairwise_matches.items():
    for tid_a, tid_b, score in matches:
        src = node_map.get((outlet_a, tid_a))
        tgt = node_map.get((outlet_b, tid_b))
        if src is not None and tgt is not None:
            link_source.append(src)
            link_target.append(tgt)
            # Value = article count in source topic
            topics_arr = np.array(outlet_results[outlet_a]['topics'])
            val = max(1, int((topics_arr == tid_a).sum()))
            link_value.append(val)
            link_color.append(f'rgba(150,150,150,0.3)')

fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        label=node_labels,
        color=node_colors,
    ),
    link=dict(
        source=link_source,
        target=link_target,
        value=link_value,
        color=link_color,
    ),
))

fig.update_layout(
    title='Cross-Outlet Topic Correspondences',
    height=700,
    width=1000,
    font_size=10,
)
fig.show()

### 7f. Unique vs Shared Topics

In [ ]:
# Stacked bar chart: shared vs unique topics per outlet
shared_unique_rows = []
for outlet in OUTLETS:
    name = SOURCE_NAMES[outlet]
    all_tids = set(get_topic_ids(outlet))

    # Topics that appear in shared groups
    shared_tids = set()
    for g in shared_groups:
        for o, tid in g['members']:
            if o == outlet:
                shared_tids.add(tid)

    unique_tids = all_tids - shared_tids
    shared_unique_rows.append({'Outlet': name, 'Type': 'Shared', 'Count': len(shared_tids)})
    shared_unique_rows.append({'Outlet': name, 'Type': 'Unique', 'Count': len(unique_tids)})

su_df = pd.DataFrame(shared_unique_rows)

fig = px.bar(
    su_df, x='Outlet', y='Count', color='Type',
    barmode='stack',
    color_discrete_map={'Shared': '#4CAF50', 'Unique': '#FF9800'},
    title='Shared vs Unique Topics per Outlet',
)
fig.update_layout(height=400)
fig.show()

# List unique topics with their keywords
print("\nUnique Topics (only in one outlet):")
print("=" * 60)
for g in unique_groups:
    outlet, tid = g['members'][0]
    words = outlet_results[outlet]['model'].get_topic(tid)
    kw = ', '.join([w for w, _ in words[:8]]) if words else 'N/A'
    topics_arr = np.array(outlet_results[outlet]['topics'])
    count = int((topics_arr == tid).sum())
    print(f"  {SOURCE_NAMES[outlet]:15s} | T{tid:2d} ({count:3d} articles) | {kw}")

### 7g. Selection Bias Metric

In [ ]:
# Rank topic groups by coverage proportion variance across outlets
bias_rows = []
for g in shared_groups:
    proportions = {}
    for outlet in OUTLETS:
        member_tids = [tid for o, tid in g['members'] if o == outlet]
        total_articles = len(outlet_results[outlet]['documents'])
        count = 0
        if member_tids:
            topics_arr = np.array(outlet_results[outlet]['topics'])
            for tid in member_tids:
                count += int((topics_arr == tid).sum())
        proportions[SOURCE_NAMES[outlet]] = count / total_articles if total_articles > 0 else 0

    props = list(proportions.values())
    variance = np.var(props)
    max_outlet = max(proportions, key=proportions.get)
    min_outlet = min(proportions, key=proportions.get)

    bias_rows.append({
        'Group': f"G{g['group_id']}",
        'Label': g['label'][:40],
        'Variance': variance,
        'Max Coverage': f"{max_outlet} ({proportions[max_outlet]:.1%})",
        'Min Coverage': f"{min_outlet} ({proportions[min_outlet]:.1%})",
        'Spread': proportions[max_outlet] - proportions[min_outlet],
        **{f"{k} %": f"{v:.1%}" for k, v in proportions.items()},
    })

bias_df = pd.DataFrame(bias_rows).sort_values('Variance', ascending=False)

print("Selection Bias Indicators (by coverage variance)")
print("=" * 80)
display(bias_df)

# Bar chart of spread
fig = px.bar(
    bias_df.head(15),
    x='Label',
    y='Spread',
    title='Topic Coverage Spread (max - min proportion) Across Outlets<br><sub>Higher spread = stronger selection bias signal</sub>',
    labels={'Spread': 'Max-Min Proportion Spread', 'Label': 'Topic Group'},
    color='Spread',
    color_continuous_scale='Reds',
)
fig.update_layout(height=450, xaxis_tickangle=-30)
fig.show()

## 8. Summary

In [ ]:
print("=" * 70)
print("PER-OUTLET TOPIC COMPARISON SUMMARY")
print("=" * 70)

# Per-outlet statistics
print("\n1. Per-Outlet Statistics")
print("-" * 50)
for outlet in OUTLETS:
    r = outlet_results[outlet]
    ti = r['topic_info']
    n_topics = len(ti[ti['Topic'] != -1])
    n_outliers = sum(1 for t in r['topics'] if t == -1)
    n_docs = len(r['documents'])
    print(f"  {SOURCE_NAMES[outlet]:15s}: {n_docs:4d} articles, {n_topics:3d} topics, "
          f"{n_outliers:3d} outliers ({100*n_outliers/n_docs:.1f}%)")

# Cross-outlet group distribution
print(f"\n2. Cross-Outlet Topic Group Distribution")
print("-" * 50)
dist = Counter(g['n_outlets'] for g in topic_groups)
for n in sorted(dist.keys(), reverse=True):
    label = 'outlet' if n == 1 else 'outlets'
    print(f"  In {n} {label}: {dist[n]} groups")

# Selection bias indicators
print(f"\n3. Selection Bias Indicators")
print("-" * 50)
if len(bias_df) > 0:
    # Per-outlet: how many unique topics
    for outlet in OUTLETS:
        unique_count = sum(1 for g in unique_groups
                          if g['members'][0][0] == outlet)
        print(f"  {SOURCE_NAMES[outlet]:15s}: {unique_count} unique topics (not matched to other outlets)")

# Most unevenly covered topics
print(f"\n4. Most Unevenly Covered Topics")
print("-" * 50)
for _, row in bias_df.head(5).iterrows():
    print(f"  {row['Group']:5s} | {row['Label']:40s} | Spread: {row['Spread']:.1%}")
    print(f"         Max: {row['Max Coverage']:30s} | Min: {row['Min Coverage']}")

print("\n" + "=" * 70)
print("Analysis complete.")